In [28]:
import pip

package_names = ['lightgbm', 'awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [29]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import LinearSVC
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier 
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import confusion_matrix
#from sklearn import metrics
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import RandomizedSearchCV

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)

478

### CONFIG

In [30]:
desfase = 1
proyecto = 'trading'
responsable = 'victor'
now = datetime.now()
sess = sagemaker.session.Session()
s3 = boto3.resource('s3')
bucket = sess.default_bucket() 
region = boto3.Session().region_name

ultimo_mes =  datetime.strptime((now - relativedelta(months=desfase)).strftime('%Y-%m'), '%Y-%m').strftime('%Y-%m-%d')
ultimo_periodo = ultimo_mes.replace('-', '')[:6]

print(bucket, '\n',ultimo_mes, '\n', ultimo_periodo)

sagemaker-us-east-1-058528764918 
 2021-05-01 
 202105


### HABILITADO RAW

In [31]:
key_origin = 'vpc/{}/{}/INPUTS/INPUT_{}.csv'.format(proyecto, responsable, ultimo_periodo)
key_share = 'vpc/{}/{}/RAW/INPUT.csv'.format(proyecto, responsable)
path_share = 's3://{}/vpc/{}/{}/RAW'.format(bucket, proyecto, responsable)
path_outputs = 's3://{}/vpc/{}/{}/OUPUT'.format(bucket, proyecto, responsable)

s3.meta.client.copy({
    'Bucket': bucket,
    'Key': key_origin
},
    bucket,
    key_share
)

print('key_origin: ', key_origin)
print('key_inputs: ', key_share)
print('path_share: ', path_share)
print('path_outputs: ', path_outputs)

key_origin:  vpc/trading/victor/INPUTS/INPUT_202105.csv
key_inputs:  vpc/trading/victor/RAW/INPUT.csv
path_share:  s3://sagemaker-us-east-1-058528764918/vpc/trading/victor/RAW
path_outputs:  s3://sagemaker-us-east-1-058528764918/vpc/trading/victor/OUPUT


In [32]:
#data = wr.s3.read_csv(path_inputs, sep=',', encoding='ISO-8859-1', dtype=dicc_str)
#print(data.shape)

### PROCESAMIENTO DESACOPLADO

In [33]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor1 = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [39]:
%%writefile read_test.py

#nativos
import random as rn
import glob
import json
import os

#calculo
import pandas as pd
import numpy as np
import scipy

#terceros
import pip

package_names = ['lightgbm']
pip.main(['install'] + package_names + ['--upgrade']) 

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

if __name__=='__main__':
    path_input = '/opt/ml/processing/input'

    for csvname in glob.iglob('{}/*.json'.format(path_input), recursive=True):
        with open(csvname, 'r') as name_file:
            _dict = json.load(name_file)
            outfile_name = name_file.replace('input', 'output')
            
            with open(outfile_name, 'w+') as outfile:
                json.dump(_dict, outfile)

Writing read_test.py


In [ ]:
processing_job_name = "trading-workflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
path_input = '/opt/ml/processing/input'
path_output = path_input.replace('input', 'output')

sklearn_processor1.run(
    code='read_test.py',
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=path_share,
            destination=path_input,
            s3_data_distribution_type='ShardedByS3Key'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=path_outputs,
            source=path_output
        )
    ]
)

preprocessing_job_description = sklearn_processor1.jobs[-1].describe()


Job Name:  trading-workflow-2021-06-25-01-34-50
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/trading/victor/RAW', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/trading-workflow-2021-06-25-01-34-50/input/code/read_test.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/trading/victor/OUPUT', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
Creating processing-job with name trading-workflow-2021-06-25-01-34-50
....

### CARGA NUEVO INPUT

In [ ]:
%%time
dicc_str = {
    'NUM_RUC': str, 
    'COD_UNICO': str, 
    'PERIODO': str, 
    'TIPDOC': str,
    'CODDOC': str
}

def tendencia_trading(lista, minimo=0):
    media_trim = np.mean(lista[-3:])
    
    if lista[-6] == -1 and lista[-5] == -1 and lista[-4] == -1 and lista[-3] == -1 and lista[-2] == -1:
        return int(lista[-1] > minimo)
    elif lista[-6] == -1 and lista[-5] == -1 and lista[-4] == -1 and lista[-3] == -1:
        return int(np.mean(lista[-2:]) > minimo)
    elif lista[-6] == -1 and lista[-5] == -1 and lista[-4] == -1:
        return int(np.mean(lista[-3:]) > minimo)
    elif lista[-6] == -1 and lista[-5] == -1:
        median_pasado = lista[-4]
    elif lista[-6] == -1:
        median_pasado = np.mean(lista[-5:-3])
    else:
        median_pasado = np.mean(lista[-6:-3])
        
    if media_trim <= minimo and median_pasado <= minimo:
        return 0
    elif median_pasado <= minimo:
        return 1.01

    return media_trim / median_pasado

def div_fix(a, b, minimo=0.001):
    if a + b == 0:
        return 0
    if a < minimo and b < minimo:
        return 0
    elif b == 0:
        return 1
    
    return a / b

def search_especific_o_menor(lista, wanted):
    meses = 0
    for val in lista[::-1]:
        if val <= wanted:
            return meses
        meses += 1
    
    return meses

def generate_listado_historico(data, col_name='listado', values='VOLUMEN_TRADING_MES', 
                               index=['COD_UNICO'], columns=['PERIODO'], fill=0,
                               aggfunc=np.sum, tramos=[]):
    
    pivoteo = pd.pivot_table(
        data, 
        values=values, index=index, columns=columns, aggfunc=aggfunc, fill_value=fill
    ).unstack().reset_index()
    
    pivoteo.columns = columns + index + [col_name]
    
    contador = 1
    dicc = {col_name: list}
    
    for tramo in tramos:
        _6_meses_previos = tramo[1]
        print(tramo[0], _6_meses_previos)
        
        _6m_actual = pivoteo[pivoteo['PERIODO'].astype(int).isin(_6_meses_previos)].groupby(by=index).agg(dicc)
        
        _6m_actual = _6m_actual.reset_index().rename(columns={'index': index[0]})
        _6m_actual['PERIODO'] = tramo[0]

        if contador == 1:
            acum_6 = _6m_actual.copy()
        else:
            acum_6 = pd.concat(
                [acum_6, _6m_actual], axis=0, ignore_index=True
            )

        print(acum_6.shape)
        contador +=1
        del _6m_actual
        
    del pivoteo
    return acum_6


if __name__=='__main__':
    print("/"*50, 'CARGA')
    data = pd.read_csv(path_inputs, sep=',', encoding='ISO-8859-1', dtype=dicc_str)
    print(data.shape)
    cols_del = [col for col in data.columns if ('TARGET' in col.upper() or 'FUTURO' in col.upper())]
    print(len(cols_del))

    print(data.shape)
    data = data.drop(cols_del, axis=1)
    print(data.shape)
    print("/"*50)


    print("/"*50, 'IMPUTACION')
    cols_id = ['COD_UNICO', 'PERIODO','CODDOC', 'NUM_RUC']
    print(len(cols_id))

    cols_num = [col for col in data.describe().T.index if col not in cols_id]
    print(len(cols_num))

    cols_str = [col for col in data.describe(include=[object, bool]).T.index if col not in cols_id]
    print(len(cols_str))

    nulls = data.isnull().sum()
    cols_nulls = nulls[nulls > 0].index

    for col in cols_nulls:

        if col in cols_num:
            data[col] = data[col].fillna(0)
            print("num: ", end=' ')

        elif col in cols_str:
            data[col] = data[col].fillna('DESCONOCIDO')
            print("cat: ", end=' ')

        print(col)
    print("/"*50)


    print("/"*50, 'VARIABLES HISTORICAS')
    tramo_actual = []
    rango_actual =  [_ for _ in list(pd.date_range(ultimo_mes, ultimo_mes, freq='MS'))]
    print(rango_actual)

    for periodo_base in rango_actual:

        rango_ = [
            int(_.strftime('%Y%m')) for _ in list(
                pd.date_range((periodo_base - relativedelta(months=8)), (periodo_base), freq='MS')
            )
        ]
        tramo_actual.append(
            (int(periodo_base.strftime('%Y%m')),           #mes cosecha
             rango_,  # listado 14 periodos sumando dos adelante y restando 11 periodos atras
             [rango_[0]],     #hace 6 meses
             [rango_[-1]])    #mes actual
        )

        print(tramo_actual[0][0], tramo_actual[0][1])
        print(tramo_actual[0][2], tramo_actual[0][3])
    print("/"*50)


    print("/"*50, 'CANTIDAD TRADING')
    acum_6 = generate_listado_historico(data, col_name='listado', values='CNT_TRADING_MES', fill=-1, tramos=tramo_actual)
    print(acum_6.shape)
    acum_6['no_habilitado_este_mes'] = acum_6['listado'].apply(lambda _: _[-1] == -1).astype(int)
    acum_6 = acum_6[acum_6['no_habilitado_este_mes'] == 0]

    del acum_6['no_habilitado_este_mes']
    print(acum_6.shape)

    acum_6['meses_habilitado_para_trading'] = acum_6['listado'].apply(lambda _: len([mes for mes in _ if mes != -1]))
    acum_6['meses_trading_consecutivo'] = acum_6['listado'].apply(lambda _: search_especific_o_menor(_, 0))
    acum_6['meses_con_trading_U9M'] = acum_6['listado'].apply(lambda _: len([mes for mes in _ if mes > 0]))
    acum_6['ratio_frecuencia_trading_U9M'] = acum_6[['meses_con_trading_U9M', 'meses_habilitado_para_trading']].apply(lambda _: div_fix(_[0], _[1]), axis=1)
    acum_6['ratio_frecuencia_consecutiva_trading_U9M'] = acum_6[['meses_trading_consecutivo', 'meses_habilitado_para_trading']].apply(lambda _: div_fix(_[0], _[1]), axis=1)
    acum_6['tiene_6_o_mas_meses_habilitado'] = acum_6['meses_habilitado_para_trading'].apply(lambda _: _ >= 6).astype(int)
    acum_6['tiene_recien_3_o menos_meses_habilitado'] = acum_6['meses_habilitado_para_trading'].apply(lambda _: _ <= 3).astype(int)
    acum_6['tiene_recien_mas_de_3_meses_habilitado'] = acum_6['meses_habilitado_para_trading'].apply(lambda _: _ > 3).astype(int)
    acum_6['cnt_trading_hist_U6M'] = acum_6['listado'].apply(lambda _: np.sum([mes for mes in _[-6:] if mes > 0]))
    acum_6['cnt_trading_hist_U3M'] = acum_6['listado'].apply(lambda _: np.sum([mes for mes in _[-3:] if mes > 0]))
    acum_6['mean_cant_trading_U6M'] = acum_6['listado'].apply(lambda _: np.mean([mes for mes in _[-6:] if mes >= 0]))
    acum_6['median_cant_trading_U6M'] = acum_6['listado'].apply(lambda _: np.median([mes for mes in _[-6:] if mes >= 0]))
    acum_6['std_cant_trading_U6M'] = acum_6['listado'].apply(lambda _: np.std([mes for mes in _[-6:] if mes >= 0]))
    acum_6['tendencia_cant_trading_trimult_over_trimpasado_U6M'] = acum_6['listado'].apply(lambda _: tendencia_trading(_[-6:]))
    del acum_6['listado']
    acum_6.to_csv('HISTORICO_CANTIDAD_TRADING_ULTIMO_PERIODO.csv', index=False)
    print("/"*50)


    print("/"*50, 'MONTO TRADING')
    acum_6 = generate_listado_historico(data, col_name='listado', values='VOLUMEN_TRADING_MES', fill=-1, tramos=tramo_actual)
    print(acum_6.shape)
    acum_6['no_habilitado_este_mes'] = acum_6['listado'].apply(lambda _: _[-1] == -1).astype(int)
    acum_6 = acum_6[acum_6['no_habilitado_este_mes'] == 0]

    del acum_6['no_habilitado_este_mes']
    print(acum_6.shape)

    acum_6['nro_meses_trading_hist_U6M'] = acum_6['listado'].apply(lambda _: len([mes for mes in _[-6:] if mes > 0]))
    acum_6['nro_meses_trading_up_100_hist_U6M'] = acum_6['listado'].apply(lambda _: len([mes for mes in _[-6:] if mes > 100]))
    acum_6['nro_meses_trading_up_1000_hist_U6M'] = acum_6['listado'].apply(lambda _: len([mes for mes in _[-6:] if mes > 1000]))
    acum_6['mean_trading_hist_U6M'] = acum_6['listado'].apply(lambda _: np.mean([mes for mes in _[-6:] if mes >= 0]))
    acum_6['median_trading_hist_U6M'] = acum_6['listado'].apply(lambda _: np.median([mes for mes in _[-6:] if mes >= 0]))
    acum_6['std_trading_hist_U6M'] = acum_6['listado'].apply(lambda _: np.std([mes for mes in _[-6:] if mes >= 0]))

    acum_6['tendencia_monto_trading_trimult_over_trimpasado'] = acum_6['listado'].apply(lambda _: tendencia_trading(_[-6:]))
    acum_6['tendencia_monto_up_100_trading_trimult_over_trimpasado'] = acum_6['listado'].apply(lambda _: tendencia_trading(_[-6:], 99.99))
    acum_6['tendencia_monto_up_1000_trading_trimult_over_trimpasado'] = acum_6['listado'].apply(lambda _: tendencia_trading(_[-6:], 999.99))

    del acum_6['listado']
    acum_6.to_csv('HISTORICO_MONTO_TRADING_ULTIMO_PERIODO.csv', index=False)
    del acum_6
    gc.collect()
    print("/"*50)


    print("/"*50, 'NIVELADO')
    periodos_cosecha = [_[0] for _ in tramo_actual]
    print(periodos_cosecha)

    data = data[data['PERIODO'].astype(int).isin(periodos_cosecha)]
    data['PERIODO'] = data['PERIODO'].astype(str)
    data['COD_UNICO'] = data['COD_UNICO'].astype(str)

    cantidades = pd.read_csv('HISTORICO_CANTIDAD_TRADING_ULTIMO_PERIODO.csv',  dtype={'COD_UNICO': str, 'PERIODO': str})

    print(data.shape)
    data = data.merge(
        cantidades, on=['COD_UNICO', 'PERIODO'], how='left'
    )
    print(data.shape)
    del cantidades
    gc.collect()

    monto = pd.read_csv('HISTORICO_MONTO_TRADING_ULTIMO_PERIODO.csv',  dtype={'COD_UNICO': str, 'PERIODO': str})

    print(data.shape)
    data = data.merge(
        monto, on=['COD_UNICO', 'PERIODO'], how='left'
    )
    print(data.shape)
    del monto
    gc.collect()
    print("/"*50)

    print("/"*50, 'CORRECCION CAMPOS NEGACION')
    data['FLG_NO_TRADING_U2M'] = data[['FLG_NO_TRADING_U3M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 2 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U3M'] = data[['FLG_NO_TRADING_U3M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 3 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U4M'] = data[['FLG_NO_TRADING_U4M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 4 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U5M'] = data[['FLG_NO_TRADING_U5M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 5 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U6M'] = data[['FLG_NO_TRADING_U6M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 6 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U7M'] = data[['FLG_NO_TRADING_U7M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 7 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U8M'] = data[['FLG_NO_TRADING_U8M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 8 and _[0] == 1 else _[0], axis=1)
    data['FLG_NO_TRADING_U9M'] = data[['FLG_NO_TRADING_U9M', 'meses_habilitado_para_trading']].apply(lambda _: 0 if _[1] < 9 and _[0] == 1 else _[0], axis=1)
    print("/"*50)

    print("/"*50, 'TRATAMIENTO DE CATEGORICOS')
    for col in cols_str:
        if col in data.columns:
            print(col)
            size = data[col].unique().shape[0]

            if size == 1:
                del data[col]

            elif size == 2:
                data = pd.get_dummies(data, columns=[col], drop_first=True)

            elif size <= 9:
                data = pd.get_dummies(data, columns=[col], drop_first=False)

    data['LIMA_CALLAO'] = data['DEPARTAMENTO'].apply(lambda _: _ in ['LIMA', 'CALLAO']).astype(int)
    data['LIMA_CALLAO_ICA_TACNA'] = data['DEPARTAMENTO'].apply(lambda _: _ in ['LIMA', 'CALLAO', 'ICA', 'TACNA']).astype(int)
    data['INMOBILIARIO_OTROS_AUTOMOTRIZ_MINERIA'] = data['SEI_MCDO'].apply(
        lambda _: _ in 'INMOBILIARIO%,%OTROS%,%AUTOMOTRIZ VEHÍCULOS%,%MINERÍA'
    ).astype(int)

    if 'FEC_INGRESO' in data.columns:
        del data['FEC_INGRESO']
    
    

### CORRECCION CAMPOS NEGACION

### TRATAMIENTO DE CATEGORICOS

In [ ]:
cols_str = [col for col in data.describe(include=[object, bool]).T.index if col not in cols_id]
print(cols_str)
len(cols_str)

In [ ]:
for col_cat in cols_str:
    with open('encoder_target_{}.json'.format(col_cat.lower()), 'r') as name_file:
        _dict = json.load(name_file)
        print(col_cat)
        
        for target in [col_target_0, col_target_100, col_target_1000]:
            name = col_cat + '_encoder_' + target.replace('TARGET_TRADING', '')
            data[name] = data[col_cat].map(_dict[target])
            print(name, end=' ')
            
        print("")

In [ ]:
data.shape

### MODELO 100

In [ ]:
cols_select_100 = [
    'nro_meses_trading_up_100_hist_U6M', 'ES_ACTIVO_S', 'VOLUMEN_TRADING_MES', 'ratio_frecuencia_trading_U9M',
    'FLG_TRADING_U3M_CONSECUTIVOS', 'CNT_CUENTAS_DOLARES', 'CNT_CUENTAS_VISTA', 'REACTIVA_IBK',
    'SALDO_CUENTA_US', 'SALDO_PROM_SOL_TOT', 'IMPORTE_SOL_TRANSF', 'CONTEO_ADEX',
    'DEPARTAMENTO_encoder__MAYOR_100', 'SEI_MCDO_encoder__MAYOR_100'
]
model_100 = joblib.load('{}/2_lgbmclassifier_20210623095243.pkl'.format(col_target_100))
print(len(cols_select_100))

In [ ]:
importancia_modelo_100 = pd.DataFrame({
    'vars': cols_select_100, 
    'imp':list(model_100.feature_importances_)
}).sort_values(
    by=['imp'], 
    ascending=False
)

sns.barplot(x="imp", y="vars", data=importancia_modelo_100, palette="mako")  #mako

### Modelo 1000

In [ ]:
cols_select_1000 = [
    'nro_meses_trading_up_1000_hist_U6M', 'FLG_TRADING_U3M_CONSECUTIVOS',
    'IMPORTE_SOL_TRANSF', 'VOLUMEN_DOL_FISICO', 'CNT_CUENTAS_VISTA', 'CNT_CUENTAS_AHORRO', 'SALDO_CUENTA_US',
    'SALDO_PROM_SOL_TOT', 'tendencia_monto_up_1000_trading_trimult_over_trimpasado',
    'MONTO_IMPORTACION', 'DEPARTAMENTO_encoder__MAYOR_1000', 'SEI_MCDO_encoder__MAYOR_1000',
    'VOLUMEN_TRADING_MES', 'ES_ACTIVO_S'
]
model_1000 = joblib.load('{}/2_lgbmclassifier_20210624013237.pkl'.format(col_target_1000))
print(len(cols_select_1000))

In [ ]:
importancia_modelo_1000 = pd.DataFrame({
    'vars': cols_select_1000, 
    'imp':list(model_1000.feature_importances_)
}).sort_values(
    by=['imp'], 
    ascending=False
)

sns.barplot(x="imp", y="vars", data=importancia_modelo_1000, palette="mako")  #mako

### PREDICCION

In [ ]:
base = data[['PERIODO', 'COD_UNICO', 'cnt_trading_hist_U6M']]
data_100 = data[cols_select_100]
data_1000 = data[cols_select_1000]

del data
gc.collect()

In [ ]:
base['SCORE_100'] = model_100.predict_proba(data_100)[:,1]
base['SCORE_1000'] = model_1000.predict_proba(data_1000)[:,1]

del data_100
del data_1000

In [ ]:
base['FRECUENCIA_0'] = base['cnt_trading_hist_U6M'].apply(lambda _: 'sin_trading' if _ == 0 else 'al_menos_1_trading')
base['FRECUENCIA_3'] = base['cnt_trading_hist_U6M'].apply(lambda _: '3_o_mas' if _ >= 3 else 'menos_de_3')

base['GRUPO_100'] = list(pd.qcut(base['SCORE_100'], cortes_deciles, labels=grupos_score))
base['GRUPO_1000'] = list(pd.qcut(base['SCORE_1000'], cortes_deciles, labels=grupos_score))

In [ ]:
base['GRUPO_CONCATENADO'] = base['GRUPO_100'] + base['GRUPO_1000']
pd.crosstab(
    base['GRUPO_100'], base['GRUPO_1000']
)

In [ ]:
base['GRUPO_FINAL'] = base['GRUPO_CONCATENADO'].apply(grupo_final)
base['GRUPO_FINAL'].value_counts()

## CRUCES CON FRECUENCIA

In [ ]:
pd.crosstab(
    base['FRECUENCIA_0'], base['GRUPO_FINAL']
)

In [ ]:
pd.crosstab(
    base['FRECUENCIA_3'], base['GRUPO_FINAL']
)

### GUARDADO BASE

In [ ]:
base['PERIODO'].max()

In [ ]:
base.to_excel('BASE_TRADING_{}.xlsx'.format(base['PERIODO'].max()), index=False)